In [8]:
#!/usr/bin/env python
# INSTRUCTIONS
# Your task for this assignment is to combine the principles that you learned 
# in unit 3, 4 and 5 and create a fully automated program that can display
# the cause-effect chain automatically.
#
# In problem set 4 you created a program that generated cause chain
# if you provided it the locations (line and iteration number) to look at.
# That is not very useful. If you know the lines to look for changes, you
# already know a lot about the cause. Instead now, with the help of concepts
# introduced in unit 5 (line coverage), improve this program to create
# the locations list automatically, and then use it to print out only the
# failure inducing lines, as before.
# See some hints at the provided functions, and an example output at the end.

import sys
import copy

# the buggy program
def remove_html_markup(s):
    tag   = False
    quote = False
    out   = ""

    for c in s:

        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out
    

# Global variables to communicate between callbacks and drivers
the_line      = None
the_iteration = None
the_state     = None
the_diff      = None
the_input     = None
coverage      = []


def traceit(frame, event, arg):
    """Tracing function that records the covered lines during 
    the execution of the program, in order of their execution, 
    and saves them in the global variable 'coverage':
    [8, 9, 10, 11, 12, 14, 16, 17, 11, 12, ...]
    """
    global coverage

    if event == "line":
        lineno   = frame.f_lineno
        
        # This check is only necessary when running this code in Jupyter notebook
        filename = frame.f_code.co_filename
        if "Anaconda" in filename:
            return traceit
        
        coverage.append(lineno)
        
    return traceit


def make_locations(coverage):
    """Returns a list of tuples in the format
    [(line, iteration), (line, iteration) ...]
    """
    locations = []
    
    # create a dictionary to record the number of iterations
    # for each line in coverage
    iterations = dict.fromkeys(coverage, 0)

    for line in coverage:
        iterations[line] += 1
        locations.append((line, iterations[line]))
    
    return locations


def trace_fetch_state(frame, event, arg):
    """Tracing function that traces the program execution until the specified
    global variables 'the_line' and 'the_iteration', and then records 
    the state of the program in the global variable 'the_state'.
    
    Complement to the 'get_state' function.
    """ 
    global the_line
    global the_iteration
    global the_state
    
    # If function is executed for the first time, create
    # function attribute 'iteration' and set it to zero
    if not hasattr(trace_fetch_state, "iteration"):
        trace_fetch_state.iteration = 0
    
    if event == "line" and frame.f_lineno == the_line:
        trace_fetch_state.iteration += 1
        if trace_fetch_state.iteration == the_iteration:
            the_state = copy.deepcopy(frame.f_locals)
            # reset 'iteration' attribute for subsequent function calls
            trace_fetch_state.iteration = 0
            # stop tracing
            sys.settrace(None)
    
    return trace_fetch_state


def get_state(input_string, line, iteration):
    """"Returns the state of the program at the specified line and iteration.
    
    Complement to the 'trace_fetch_state' function.
    """
    global the_line
    global the_iteration
    global the_state
    
    the_line      = line
    the_iteration = iteration
    
    sys.settrace(trace_fetch_state)
    y = remove_html_markup(input_string)
    sys.settrace(None)
    
    return the_state


def test(diffs):
    """Testing function that calls 'remove_html_markup', stops
    at 'the_line' and 'the_iteration' and applies the differences in 'diffs'.
    Finally, it checks the returned value of 'remove_html_markup' and
    returns "PASS" or "FAIL" accordingly.
    
    Complement to the 'trace_apply_diff' function.
    """
    global the_diff
    global the_input
    global the_line
    global the_iteration

    line      = the_line
    iteration = the_iteration
    
    the_diff = diffs
    sys.settrace(trace_apply_diff)
    y = remove_html_markup(the_input)
    sys.settrace(None)

    the_line      = line
    the_iteration = iteration

    if y.find('<') == -1:
        return "PASS"
    else:
        return "FAIL"


def ddmin(s):
    """Delta debugger. 
    
    It is used to minimize the list of variable values that cause the failing run.
    """

    n = 2     # Initial granularity
    while len(s) >= 2:
        start = 0
        subset_length = len(s) / n
        some_complement_is_failing = False

        while start < len(s):
            complement = s[:start] + s[start + subset_length:]

            if test(complement) == "FAIL":
                s = complement
                n = max(n - 1, 2)
                some_complement_is_failing = True
                break
                
            start += subset_length

        if not some_complement_is_failing:
            if n == len(s):
                break
            n = min(n * 2, len(s))

    return s


def trace_apply_diff(frame, event, arg):
    """Tracing function that stops at 'the_line' and 'the_iteration'
    and updates the execution frame's variable dictionary 'frame.f_locals',
    with the values found in global variable 'the_diff'.
    
    Complement to the 'test' function.
    """
    global the_line
    global the_diff
    global the_iteration

    if frame.f_lineno == the_line:
        the_iteration = the_iteration - 1
        if the_iteration == 0:
            frame.f_locals.update(the_diff)
            the_line = None
            return None  # Stop tracing
    
    return trace_apply_diff


def auto_cause_chain(locations):
    """Iterates over all the (line, iteration) pairs provided.
    
    - Then, for each line and iteration pair, it compares the various 
    variable values between the corresponding failing and passing runs 
    and calculates a list of differences of the form:
    [(variable1 name, variable1 value at failing run), ...]
    
    - This list is then passed to 'ddmin' which does the delta
    debugging and returns the minimum set of failure-inducing 
    variables from the list.
    """
    global html_fail
    global html_pass
    global the_input
    global the_line
    global the_iteration
    global the_diff
    
    print "The program was started with", repr(html_fail)
    
    """This is the list of causes found while checking each (line, iteration).
    It is used to avoid duplicating the same cause in the output.
    This is necessary because the program variables state 
    may not change between two executed lines and thus the 
    differences variable 'diffs' will be exactly the same."""
    causes = []
    
    # Main loop: test for each covered line and iteration
    for (line, iteration) in locations:

        # Get the passing and the failing states
        state_pass = get_state(html_pass, line, iteration)
        state_fail = get_state(html_fail, line, iteration)
    
        # Compute the differences
        diffs = []
        # First check if the execution frame's variable dictionary is not empty
        if state_fail:
            for var in state_fail:
                if (var not in state_pass) or state_pass[var] != state_fail[var]:
                    diffs.append((var, state_fail[var]))
        
        # Minimize the failure-inducing set of differences
        # Since this time you have all the covered lines and iterations in
        # locations, you will have to figure out how to automatically detect
        # which lines/iterations are the ones that are part of the
        # failure chain and print out only these.
        the_input = html_pass
        the_line  = line
        the_iteration  = iteration
        
        if diffs:
            cause = ddmin(diffs)
            if cause not in causes:
                causes.append(cause)
                for variable, value in cause:
                    print 'Then', variable, 'became', repr(value)
            
    print "Then the program failed."


###### Testing runs
# We will test your function with different strings and on a different function      
html_fail = '"<b>foo</b>"'
html_pass = "'<b>foo</b>'"


# MAIN PROGRAM:

# Record line coverage of the failing run in global variable 'coverage'
sys.settrace(traceit)
remove_html_markup(html_fail)
sys.settrace(None)

# Create a list of tuples in the format [(line1, iteration1), ...] using
# the line coverage data from 'coverage'
locations = make_locations(coverage)

# Execute main function 'auto_cause_chain'
auto_cause_chain(locations)


# The output should look like follows:
"""
The program was started with '"<b>foo</b>"'
Then s became '"<b>foo</b>"'
Then c became '"'
Then quote became True
...
"""


The program was started with '"<b>foo</b>"'
Then s became '"<b>foo</b>"'
Then c became '"'
Then quote became True
Then tag became False
Then c became '<'
Then c became '<'
Then out became '<'
Then out became '<b'
Then out became '<b>'
Then out became '<b>f'
Then out became '<b>fo'
Then c became '<'
Then out became ''
Then out became '<b>foo'
Then c became 'b'
Then out became '<'
Then out became '<b>foo<'
Then quote became False
Then c became '"'
Then out became ''
Then c became '>'
Then out became '<b'
Then out became '<b>foo</'
Then c became 'f'
Then out became '<b>'
Then out became '<b>foo</b'
Then quote became True
Then c became 'b'
Then out became '<'
Then c became 'o'
Then out became '<b>f'
Then out became '<b>foo</b>'
Then c became '"'
Then out became '<b>foo</b>'
Then the program failed.


'\nThe program was started with \'"<b>foo</b>"\'\nThen s became \'"<b>foo</b>"\'\nThen c became \'"\'\nThen quote became True\n...\n'